In [1]:
import numpy as np
import scipy.linalg

In [2]:
N = 3

u = np.random.randn(N) ** 2
v = np.random.randn(N) ** 2

A = np.random.randn(N, N)
A = 10 * np.eye(N) + A @ A.T

L = np.linalg.cholesky(A)

A_prime = (np.eye(N) + np.outer(v, u)) @ A @ (np.eye(N) + np.outer(u, v))

In [3]:
A, L

(array([[11.26183562, -0.38452658,  1.74919203],
        [-0.38452658, 10.63368655, -1.61248075],
        [ 1.74919203, -1.61248075, 15.07942133]]),
 array([[ 3.35586585,  0.        ,  0.        ],
        [-0.11458342,  3.25891964,  0.        ],
        [ 0.52123419, -0.47646341,  3.81847075]]))

In [4]:
A_prime

array([[ 13.13353263,   4.56364762,  65.13175181],
       [  4.56364762,  12.25523324,  27.30645806],
       [ 65.13175181,  27.30645806, 484.08806673]])

In [5]:
z = scipy.linalg.solve_triangular(L, v, lower=True)
w = scipy.linalg.solve_triangular(L, A @ u, lower=True)

In [6]:
z

array([0.00595764, 0.03160829, 0.33717848])

In [7]:
w

array([13.25685082, -0.74692641,  7.47195607])

In [8]:
beta_bar = np.zeros(N)
s = np.zeros(N)

beta_bar[-1] = 1.0 / w[-1]

for j in range(N - 1 - 1, -1, -1):
    r = beta_bar[j + 1] * w[j]
    s_j = 1.0 / np.sqrt(r ** 2 + 1)
    c = r * s_j
    s[j] = s_j
    beta_bar[j] = s[j] * beta_bar[j + 1]
    beta_bar[j + 1] *= c

beta_bar

array([ 0.06563453,  0.11587221, -0.01331221])

In [9]:
Q_0 = np.triu(beta_bar[:, None] * w[None, :]) + np.diag(-s[:-1], -1)
Q_0

array([[ 0.87010718, -0.04902416,  0.49041833],
       [-0.49286256, -0.08654802,  0.86579209],
       [ 0.        , -0.99504074, -0.09946823]])

In [10]:
H_bar = Q_0.T.copy()
H_bar[:, 0] += (1.0 / beta_bar[0]) * z
H_bar

array([[ 0.96087706, -0.49286256,  0.        ],
       [ 0.43255609, -0.08654802, -0.99504074],
       [ 5.62763009,  0.86579209, -0.09946823]])

In [15]:
L_bar = H_bar.copy(order="F")

for j in range(N - 1):
    c, s = scipy.linalg.blas.drotg(L_bar[j, j], L_bar[j, j + 1])
    
    x, y = scipy.linalg.blas.drot(x=L_bar[j, j], y=L_bar[j, j + 1], c=c, s=s)
    
    if x < 0:
        c = -c
        s = -s
    
    scipy.linalg.blas.drot(
        x=L_bar[j:, j],
        y=L_bar[j:, j + 1],
        c=c,
        s=s,
        overwrite_x=True,
        overwrite_y=True,
    )
    
L_bar = np.tril(L_bar)

In [16]:
L_prime = L @ L_bar

In [17]:
L_prime

array([[ 3.62402161,  0.        ,  0.        ],
       [ 1.25927715,  3.26641306,  0.        ],
       [17.97223053,  1.43106174, 12.6110689 ]])

In [19]:
np.linalg.cholesky(A_prime) - L_prime

array([[-4.44089210e-16,  0.00000000e+00,  0.00000000e+00],
       [-4.44089210e-16,  4.44089210e-16,  0.00000000e+00],
       [ 0.00000000e+00,  1.33226763e-15,  1.77635684e-15]])

In [ ]:
L_prime @ L_prime.T

In [ ]:
A_prime

In [ ]:
np.linalg.cholesky(A_prime) @ np.linalg.cholesky(A_prime).T